In [1]:
import logging
import re
import os
os.chdir('../code/')
import random

import torch
from torch.utils.data import DataLoader
# from tqdm import tqdm, trange
from tqdm.notebook import tqdm, trange
from transformers import (AdamW, AutoModelForSeq2SeqLM,
                          AutoTokenizer, get_linear_schedule_with_warmup)


In [2]:
from constants import *
from data_utils import (ABSADataset, filter_none, filter_invalid,
                        get_dataset, get_inputs, normalize_augment)
from model_utils import (prepare_constrained_tokens, prepare_tag_tokens)
from main import *
from data_utils import *

In [5]:
# Load arguments from the JSON file

import json
import argparse

arg_path = '../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/args.json'
with open(arg_path, 'r') as file:
    args_dict = json.load(file)

# Create a namespace from the dictionary
args = argparse.Namespace(**args_dict)

args.model_name_or_path = '../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/checkpoint-e24'
print(args)


Namespace(adam_epsilon=1e-08, beam=1, clear_model=True, commit=None, data_dir='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/data', data_gene=True, data_gene_aug_num=None, data_gene_aug_ratio=None, data_gene_decode=None, data_gene_epochs=25, data_gene_extract=True, data_gene_extract_epochs=25, data_gene_extract_none_remove_ratio=0, data_gene_min_length=0, data_gene_none_remove_ratio=0, data_gene_none_word_num=1, data_gene_num_beam=1, data_gene_same_model=False, data_gene_top_p=0.9, data_gene_wt_constrained=True, dataset='cross_domain', device='cuda', do_eval=True, do_train=True, eval_batch_size=16, extract_model=None, gene_model=None, gradient_accumulation_steps=2, inference_dir='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/inference', init_tag='english', learning_rate=0.0003, max_seq_length=128, model_filter=True, model_filter_skip_none=False, model_name_or_path='../outputs/aste/cross_domain/run_aste/seed-42/laptop14-rest14/checkpoint-e24', n_gpu='0', 

In [6]:
def openfile(path: str):
    with open(path, 'r') as file:
        lines = file.readlines()
    return [line.replace('\n','') for line in lines]


def input_output_pair(text:str):
    input, output = text.split('===>')
    return (input, output)

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path).to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=False)

In [8]:
txt_file_path = '../thai_data/laptop14/th/googletrans/train_aste_processed.txt'
row_list = openfile(txt_file_path)

input_list, target_list = [],[]
for row in tqdm(row_list):
    input, target = input_output_pair(row)
    input_list.append(input)
    target_list.append(target)



  0%|          | 0/906 [00:00<?, ?it/s]

In [10]:
input_list

['ฉันชาร์จมันในเวลากลางคืนและข้ามสายไฟติดตัวไปด้วยเพราะอายุการใช้งานแบตเตอรี่ที่ดี ',
 'มันมีคุณภาพสูงมี GUI นักฆ่ามีความเสถียรมากสามารถขยายได้สูงมีการใช้งานที่ดีมากใช้งานง่ายใช้งานง่ายและงดงามอย่างแน่นอน ',
 'ง่ายต่อการเริ่มต้นและไม่ร้อนเกินไปเท่าแล็ปท็อปอื่น ๆ ',
 'แล็ปท็อปที่ยอดเยี่ยมที่มีคุณสมบัติที่ยอดเยี่ยมมากมาย! ',
 'คืนหนึ่งฉันปิดสิ่งที่น่าประหลาดใจหลังจากใช้มันในวันถัดไปฉันเปิดเครื่องไม่มี GUI หน้าจอมืดทั้งหมดไฟคงที่ไฟฮาร์ดไดรฟ์คงที่และไม่กระพริบเหมือนปกติ ',
 'อย่างไรก็ตามท่าทางแบบมัลติทัชและพื้นที่ติดตามขนาดใหญ่ทำให้ไม่จำเป็นต้องใช้เมาส์ภายนอก (เว้นแต่คุณจะเล่นเกมอีกครั้ง) ',
 'ฉันชอบวิธีที่ชุดซอฟต์แวร์ทั้งหมดทำงานร่วมกัน ',
 'ความเร็วนั้นเหลือเชื่อและฉันก็พอใจมากกว่า ',
 'ฉันแทบจะไม่สามารถใช้อุปกรณ์ USB ใด ๆ ได้เพราะพวกเขาจะไม่เชื่อมต่ออย่างถูกต้อง ',
 'ในที่สุดเมื่อฉันมีทุกอย่างที่ทำงานกับซอฟต์แวร์ทั้งหมดของฉันที่ติดตั้งฉันเสียบอยู่ในหุ่นยนต์ของฉันเพื่อเติมเงินและระบบล่ม ',
 'การจับคู่กับ iPhone เป็นความสุขที่บริสุทธิ์ - พูดคุยเกี่ยวกับการซิงค์ที่ไม่เจ็บปวด - ใช้เพื่อพาฉั

In [19]:
dataset = ABSADataset(args, tokenizer, inputs=input_list[:10], targets=target_list)


c:\Workspace\Tutorial\BGCA\.conda\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
def infer_new(args, dataset, model, tokenizer, name=None, is_constrained=False, constrained_vocab=None, keep_mask=False, **decode_dict):
    dataloader = DataLoader(dataset, batch_size=args.eval_batch_size, num_workers=4)

    if keep_mask:
        # can't skip special directly, will lose extra_id
        unwanted_tokens = [tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token]
        unwanted_ids = tokenizer.convert_tokens_to_ids(unwanted_tokens)
        def filter_decode(ids):
            ids = [i for i in ids if i not in unwanted_ids]
            tokens = tokenizer.convert_ids_to_tokens(ids)
            sentence = tokenizer.convert_tokens_to_string(tokens)
            return sentence

    # inference
    inputs, outputs, targets = [], [], []
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Generating'):
            if is_constrained:
                prefix_fn_obj = Prefix_fn_cls(tokenizer, constrained_vocab, batch['source_ids'].to(args.device))  # need fix
                prefix_fn = lambda batch_id, sent: prefix_fn_obj.get(batch_id, sent)
            else:
                prefix_fn = None

            outs_dict = model.generate(input_ids=batch['source_ids'].to(args.device),
                                        attention_mask=batch['source_mask'].to(args.device),
                                        max_length=128,
                                        prefix_allowed_tokens_fn=prefix_fn,
                                        output_scores=True,
                                        return_dict_in_generate=True,
                                        **decode_dict,
                                        )
            outs = outs_dict["sequences"]

            if keep_mask:
                input_ = [filter_decode(ids) for ids in batch["source_ids"]]
                dec = [filter_decode(ids) for ids in outs]
                target = [filter_decode(ids) for ids in batch["target_ids"]]
            else:
                input_ = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["source_ids"]]
                dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
                target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

            inputs.extend(input_)
            outputs.extend(dec)
            targets.extend(target)

    # decode_txt = "constrained" if is_constrained else "greedy"
    # with open(os.path.join(args.inference_dir, f"{name}_{decode_txt}_output.txt"), "w") as f:
    #     for i, o in enumerate(outputs):
    #         f.write(f"{inputs[i]} ===> {o}\n")

    # return inputs, outputs, targets
    return inputs, outputs


In [20]:
input_infer, output_infer = infer_new(
        args, dataset, model, tokenizer, 
        # name=f"thai-pred",
        # is_constrained=True, 
        is_constrained=False, 
        constrained_vocab=prepare_constrained_tokens(tokenizer, args.task, args.paradigm),
    )


for x,y in zip(input_infer, output_infer):
    print(f'input text:{x}')
    print(f'predict(gene) text:{y}')

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

input text:ฉันชาร์จมันในเวลากลางคืนและข้ามสายไฟติดตัวไปด้วยเพราะอายุการใช้งานแบตเตอรี่ที่ดี
predict(gene) text:<neu> battery <opinion> bad
input text:มันมีคุณภาพสูงมี GUI นักฆ่ามีความเสถียรมากสามารถขยายได้สูงมีการใช้งานที่ดีมากใช้งานง่ายใช้งานง่ายและงดงามอย่างแน่นอน
predict(gene) text:<pos> GUI <opinion> คุณภาพสูง <pos> GUI <opinion> бережs of <pos> use <opinion> easy
input text:ง่ายต่อการเริ่มต้นและไม่ร้อนเกินไปเท่าแล็ปท็อปอื่น ๆ
predict(gene) text:<pos> เริ่มต้น <opinion> ง่าย <pos> start <opinion> ง่าย
input text:แล็ปท็อปที่ยอดเยี่ยมที่มีคุณสมบัติที่ยอดเยี่ยมมากมาย!
predict(gene) text:<pos> คุณสมบัติ <opinion> ที่ยอดเยี่ยม
input text:คืนหนึ่งฉันปิดสิ่งที่น่าประหลาดใจหลังจากใช้มันในวันถัดไปฉันเปิดเครื่องไม่มี GUI หน้าจอมืดทั้งหมดไฟคงที่ไฟฮาร์ดไดรฟ์คงที่และไม่กระพริบเหมือนปกติ
predict(gene) text:<neg> GUI <opinion> น่าประหลาดใจ <neg> GUI <opinion> มืดทั้งหมดไฟ <opinion> hard <neg> ไฟ <opinion> horrible
input text:อย่างไรก็ตามท่าทางแบบมัลติทัชและพื้นที่ติดตามขนาดใหญ่ทําให้ไม่จําเป็นต้อ